# Data Extraction using Ollama and Mixtral 7x8b

Hardware: Apple M3 Max - 64GB RAM

1. I installed the models mixtral:7x8b and mixtral:8x22b in terminal and then I tried them in python using the following code. <br><br>
2. Any variation in prompt leads to different results. To extract information from text, it must be explicitly mentioned in the prompt that the model should look for answers only in the text. However, it still made some inferences on its own, without seeing the explicit keyword for a property. As an example, being dishwasher_safe is a property that should be explicitly mentioned in the product description, however, with some materials, the model inferred them to be dishwasher_safe, things like glass as an example. However that is not what needed and need to be further refined.<br><br>
3. An example also helps with further refinement of the output of the model. with explicit keys, the model looks for specific properties in text that is needed for that product group. This may change from one product category to the next. However, product description text for each product is not standard with all the needed information. Therefore, some items need to be null or None, and we should also ask the model that when it cannot find the answers in the model, consider the value for the keys as None. It is also important to ask the model to only use keys given in the example to get uniform results. However, the model may sometimes diverge from the instruction, but the majority of responses will be uniformed.

In [ ]:
# !pip install pandas ollama

In [ ]:
import pandas as pd
import json
from values import *
import ollama
import warnings
warnings.filterwarnings('ignore')
# pd.set_option('display.max_colwidth', 50)

In [ ]:
val = Values()

### DATA CLEANING


In [ ]:
artikel_df = pd.read_excel(val.shop_file_path)
marketing_artikel = pd.read_csv(val.marketing_artikel,encoding='latin-1', delimiter=';', on_bad_lines='skip',parse_dates=val.dates,dayfirst=True)

In [ ]:
marketing_artikel = marketing_artikel[marketing_artikel['WM'].isna()==True]

In [ ]:
### Cleaning and preparing Warengroup data
warengrps = marketing_artikel[['NUMMER','WARENGR']]
warengrps['WARENGR'] = pd.to_numeric(warengrps['WARENGR'],errors='coerce')
warengrps.dropna(subset='WARENGR',inplace=True)
warengrps['WARENGR'] = warengrps['WARENGR'].astype(int)
warengrps.drop_duplicates(subset='NUMMER',inplace=True)

### Selecting a list of columns from our artikels that we need
artikels = artikel_df[['StoreId','Name','Beschreibung']]

### Cleaning up the Artikelnumbers, renaming their column and removing unwanted information to connect this dataset with Warengroups data
artikels['Number'] = artikels['StoreId'].str.split().str[0]

### Connecting two datasets to find the warengroups of each artikel and select some of them for further analysis
artikels_mit_wrgp = pd.merge(artikels,warengrps,how='left',left_on='Number',right_on='NUMMER')

### Checking the items from right dataset and see if the left-join left some rows null and removing them
artikels_mit_wrgp = artikels_mit_wrgp[artikels_mit_wrgp['NUMMER'].isna()==False]
### Converting WARENGR column values from float to int
artikels_mit_wrgp['WARENGR'] = artikels_mit_wrgp['WARENGR'].astype(int)

In [ ]:
### Selecting certain Warengroups and columns for further analysis
kuchen_gerate = artikels_mit_wrgp[artikels_mit_wrgp['WARENGR'].isin([1203])]

### Selecting the needed columns and uniforming the column names
kuchen_gerate = kuchen_gerate[['WARENGR','NUMMER','Name','Beschreibung']]
kuchen_gerate = kuchen_gerate.rename(columns={'Name':'NAME','Beschreibung':'BESCHREIBUNG'})

In [ ]:
kuchen_gerate

In [ ]:
### Removing rows without description
for id,item in enumerate(kuchen_gerate['BESCHREIBUNG']):
    kuchen_gerate = kuchen_gerate.dropna(subset='BESCHREIBUNG')

kuchen_gerate['ORIGINAL_BESCHREIBUNG'] = kuchen_gerate['BESCHREIBUNG'].copy()

In [ ]:
### Removing unwanted characters and duplicates
kuchen_gerate['BESCHREIBUNG'] = kuchen_gerate['BESCHREIBUNG'].str.replace(' ',' ',regex=True)
kuchen_gerate['BESCHREIBUNG'] = kuchen_gerate['BESCHREIBUNG'].str.replace(r'<p>|</p>|<ul>|</ul>|<li>|</li>|<br>|<b>|</b>|<span.*>|<font.*>|<strong>|</strong>',' ',regex=True,case=False)
kuchen_gerate['BESCHREIBUNG'] = kuchen_gerate['BESCHREIBUNG'].str.replace(r'&nbsp_|&nbsp;',' ',regex=True)
kuchen_gerate['BESCHREIBUNG'] =kuchen_gerate['BESCHREIBUNG'].str.replace('&Oslash',' ')
kuchen_gerate = kuchen_gerate.drop_duplicates()


In [ ]:
# Choosing certain columns and combining the name and Beschreibung as they both have product properties
mined_text = kuchen_gerate[['NUMMER','NAME','BESCHREIBUNG','ORIGINAL_BESCHREIBUNG','WARENGR']].copy()
mined_text['BESCHREIBUNG'] = mined_text['NAME'] + '\n' +mined_text['BESCHREIBUNG']
mined_text['BESCHREIBUNG'] = mined_text['BESCHREIBUNG'].str.lstrip()
mined_text['BESCHREIBUNG'] = mined_text['BESCHREIBUNG'].str.rstrip()
mined_text.reset_index(inplace=True)
mined_text


## PROMPT ENGINEERING

In [ ]:
## JSON format for the response of the model
example = {"NUMMER": "228Z01",
            "NAME": "Mini-Fritteuse", 
            "ART":"None", 
            "MATERIAL":["Aluminium","Edelstahl"], 
            "MASSEN": r"20 x 12 x 10 cm",   
            "VOLUME": r"2 l", 
            "GEWICHT":r"9 kg",
            "VOLTAGE": r"230V", 
            "WATT": r"1000W",
            "FARBE": "Rot",
            "SET": r"2-er Set",
            "OFENFEST": "Ja",
            "SPUELMACHINEFEST":"Nein",
            "BRAND":"Hagen Grote Exklusiv",
            "BESCHICHTUNG": "Antihaftbeschichtung",
            "TEMPERATUR": "bis 70 °C",
            "LED_ANZEIGE":"Ja",
            "KABEL": "1 m",
            "RUTSCHFEST": "False",
            "ANWENDUNG": "Frittieren von Pommes Frites, Meeresfrüchten, Fisch und Gemüse",
            "ANDERE_EIGENSCHAFTEN": ["kippbarer Motorkopf", "gummigelagerte Füße"]}

In [ ]:
# allowed_keys = ["NUMMER", "NAME", "ART", "MATERIAL", "MASSEN", "VOLUME", "GEWICHT", "VOLTAGE", "WATT", "FARBE", "SET", "OFENFEST", "SPUELMACHINEFEST", "BRAND", "BESCHICHTUNG", "TEMPERATUR", "LED_ANZEIGE", "KABEL", "RUTSCHFEST", "ANWENDUNG", "ANDERE_EIGENSCHAFTEN"]

# prompt = f"""[INST]Bitte extrahieren Sie die Produktattribute aus dem folgenden Text und geben Sie diese als ein gültiges JSON-Objekt in deutscher Sprache aus. Verwenden Sie nur die folgenden Schlüssel und ändern Sie diese nicht: {allowed_keys}.
# Beschränken Sie die Werte auf maximal 50 Zeichen. Wenn Sie eine Information nicht finden können, lassen Sie den entsprechenden Wert einfach leer. Ziehen Sie keine Schlüsse aus dem Text. Verwenden Sie nur das explizite Wissen, das im Text angegeben ist.

# Beachten Sie, dass das JSON-Objekt mit '{{' beginnt und mit '}}' endet. Verwenden Sie ':' (Doppelpunkt) zur Trennung von Schlüssel und Wert, und ',' (Komma) zur Trennung von Paaren. Achten Sie darauf, Anführungszeichen ('"') um die Schlüssel und Werte zu setzen.

# Beispiel-RESPONSE: {json.dumps(example, ensure_ascii=False)}
# [/INST]

# ProduktID: {id}
# Produktname: {name}
# Beschreibung: {description}
# """


## DATA EXTRACTION WITH MIXTRAL 

In [ ]:
def fix_json_response(response_text):
    # Remove extra commas at the end of the JSON object
    response_text = response_text.rstrip(',')

    # Add missing quotes around keys and values
    response_text = response_text.replace(': ', ': "').replace(', ', '", "').replace('{', '{"').replace('}', '"}')

    try:
        response_dict = json.loads(response_text)
        return response_dict
    except json.JSONDecodeError:
        print("Error: The response is still not a valid JSON object after fixing.")
        return None

In [ ]:
import json


def clean_response(response_text):
    try:
        response_dict = json.loads(response_text)
        cleaned_dict = {key: value.strip() if isinstance(value, str) else value for key, value in response_dict.items()}
        return cleaned_dict

    except json.JSONDecodeError:
        print("Error: The response is not a valid JSON object.")
        print(response_text)
        return None
    
    
for id,name, description in zip(mined_text['NUMMER'],mined_text['NAME'],mined_text['BESCHREIBUNG']):
    print(id,name)
    allowed_keys = ["NUMMER", "NAME", "ART", "MATERIAL", "MASSEN", "VOLUME", "GEWICHT", "VOLTAGE", "WATT", "FARBE", 
                    "SET", "OFENFEST", "SPUELMACHINEFEST", "BRAND", "BESCHICHTUNG", "TEMPERATUR", "LED_ANZEIGE", 
                    "KABEL", "RUTSCHFEST", "ANWENDUNG", "ANDERE_EIGENSCHAFTEN"]

    prompt = f"""[INST]Bitte extrahieren Sie die Produktattribute aus dem folgenden 
        Text und geben Sie diese als ein gültiges JSON-Objekt in deutscher Sprache aus.
        Verwenden Sie nur die folgenden Schlüssel und ändern Sie diese nicht: {allowed_keys}.
        Beschränken Sie die Werte auf maximal 50 Zeichen. Wenn Sie eine Information nicht finden können, 
        lassen Sie den entsprechenden Wert einfach leer. Ziehen Sie keine Schlüsse aus dem Text. 
        Verwenden Sie nur das explizite Wissen, das im Text angegeben ist.

        Beachten Sie, dass das JSON-Objekt mit '{{' beginnt und mit '}}' endet. Verwenden Sie ':' 
        (Doppelpunkt) zur Trennung von Schlüssel und Wert, und ',' (Komma) zur Trennung von Paaren. 
        Achten Sie darauf, Anführungszeichen ('"') um die Schlüssel und Werte zu setzen.

        Beispiel-RESPONSE: {json.dumps(example, ensure_ascii=False)}
        [/INST]

        ProduktID: {id}
        Produktname: {name}
        Beschreibung: {description}
        """

# Generate response using Mixtral 7x8b
    response = ollama.generate(model='mixtral:latest', prompt=prompt)

# Clean up the response
    cleaned_response = clean_response(response['response'])

# Save the cleaned response to a JSON file (if not None)
    if cleaned_response is not None:
        with open('Exports/data_1.json', 'a', encoding="utf-8") as f:
            json.dump(cleaned_response, f, indent=4, ensure_ascii=False)
